<a href="https://colab.research.google.com/github/igortereshchenko/python/blob/master/%D0%A2%D0%B5%D0%BE%D1%80%D0%B5%D0%BC%D0%B0_%D0%BF%D1%80%D0%BE_%D0%B3%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D1%8E_%D1%81%D0%BA%D0%BB%D0%B0%D0%B4%D0%BD%D0%BE%D1%97_%D1%84%D1%83%D0%BD%D0%BA%D1%86%D1%96%D1%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
  '''))
  init_notebook_mode(connected=False)

In [0]:
get_ipython().events.register('pre_run_cell', enable_plotly_in_cell)

In [0]:
  

import numpy as np
from plotly.offline import iplot
import plotly.graph_objs as go



y=np.linspace(0,np.pi,100)
z=np.sin(y)+1
x=np.zeros(len(z))


trace = go.Scatter3d(
    x=x, y=y, z=z,
    marker=dict(
        size=0,
    ),
    line=dict(
        color='#1f77b4',
        width=4
    )
)

data = [trace]


layout = go.Layout(
    autosize=True,


    scene = dict(
                    xaxis = dict(
                                  nticks=4, 
                                  range = [0,10],
                                ),
                    yaxis = dict(
                                  nticks=4, 
                                  range = [0,10],
                                ),
                    zaxis = dict(
                                  nticks=4, 
                                  range = [0,10],
                        tickmode='linear',
        ticks='outside',
        tick0=0,
        dtick=0.25,
        ticklen=8,
        tickwidth=4,
        tickcolor='#000'
                                ),
                  ),
#                     width=900,
                    height=800,
                    
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)


In [0]:
import numpy as np
from plotly.offline import iplot
import plotly.graph_objs as go



def function(x):
  return x*x

def createVerticalLine(y0,y1,x0,visibility=False,label="x",color = ('rgb(205, 12, 24)'), dash = None,legend=False, width=1):
  
  line ={
              "color" : color,
              "width" : width}
  
  if dash:
    line["dash"] = dash
  
  vertical_trace = go.Scatter( 
                                  x=[x0, x0],
                                  y=[y0, y1], 

                                  line = line,
                                  name=label,
                                  visible=visibility,
                                  showlegend=legend
                                )
  return vertical_trace





def createHorizontalLine(x0,x1,y0,visibility=False,label="y",color = ('rgb(12, 205, 24)'), dash = None,legend=False, width=1):
  
  line ={
              "color" : color,
              "width" : width}
  
  if dash:
    line["dash"] = dash
  
  horizontal_trace = go.Scatter( 
                                  x=[x0, x1],
                                  y=[y0, y0], 

                                  line = line,
                                  name=label,
                                  visible=visibility,
                                  showlegend=legend
                                )
  return horizontal_trace




  
def createSurroundings(x, delta, visibility):

  vertical_lines = []
  horizontal_lines = []
  

  
  for i in [-1,1]:
    vertical_lines.append(createVerticalLine(0,function(x+i*delta),x+i*delta,visibility,"x0-"+str(delta),dash="dash"))
    horizontal_lines.append(createHorizontalLine(0,x+i*delta,function(x+i*delta),visibility,"y0+"+str(delta),dash="dash"))
    


  return vertical_lines, horizontal_lines, [createHorizontalLine(x-delta,x+delta,0,width=4,color=('rgb(205, 12, 24)'),legend=True, label="delta")], [createVerticalLine(function(x-delta),function(x+delta),0,width=4,color=('rgb(12, 205, 24)'),legend=True, label="epsilon")]



def createSurroundingsList(x,delta,steps):
  vertical_lines = []
  horizontal_lines = []
  
  horizontal_surroundings = []
  vertical_surroundings = []
  for step in range(1,steps+1):
    delta_step= (delta/steps)*step
    v,h, h_s, v_s = createSurroundings(x, delta_step,visibility=False)
    vertical_lines.extend(v)
    horizontal_lines.extend(h)
  
    horizontal_surroundings.extend(h_s)
    vertical_surroundings.extend(v_s)
    
  return vertical_lines, horizontal_lines, horizontal_surroundings, vertical_surroundings



def createSteps(x0,x1,a,point_count,delta,steps):
  x = np.linspace(x0,x1,point_count)
  y = function(x)

  function_trace = go.Scatter( x=x,
                               y=y, 
                               name='f(x)',
                               visible=True

                             )
  b = function(a)
  a_trace = createVerticalLine(0,b,a,visibility=True, label="a="+str(a), color=('rgb(205, 150, 24)'),legend=True,dash="dash")
  b_trace = createHorizontalLine(0,a,b,visibility=True,label="b="+str(b),color = ('rgb(205, 150, 24)'),legend=True,dash="dash")
  v, h, h_s, v_s = createSurroundingsList(a,delta,steps)
  
  data = [function_trace,a_trace,b_trace]+v+h +h_s+v_s
  
  slider_steps=[]
  
  for i in range(steps):
    
    line_visible = [False]*2*steps
    
    line_visible[2*i]=True
    line_visible[2*i+1]=True
    
    s_visible= [False]*steps
    s_visible[i]=True
    
    step = dict(
                method='restyle',
                args=[
                    'visible', [True, True,True]+line_visible+line_visible+s_visible+s_visible
                ],
                )
    slider_steps.append(step)
  
  return data, slider_steps

  

x0 =1
x1 = 4
a = 3
count_of_points = 100
delta = 0.5
steps =10

data, slider_steps =createSteps(x0,x1,a,count_of_points,delta,steps)

sliders = dict(
    active = 0,
    currentvalue = {"prefix": "Delta: "},
    pad = {"t": 0},
    steps=slider_steps
)

layout = dict(
    
    legend=dict(
        orientation="h",
        x=0,
        y=1,
    ),
    sliders=[sliders],
    xaxis=dict(
        range=[0, x1+1],
    ),
    yaxis=dict(
        range=[0, function(x1)],
    ),
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)